In [0]:
dbutils.widgets.text("Arrival_date","2024_07_25")
start_date= dbutils.widgets.get("Arrival_date")


In [0]:
customer_data=f"/Volumes/workspace/default/scd/Customer/customer_{start_date}.csv"
print(customer_data)
df=spark.read.csv(customer_data,header=True,inferSchema=True,multiLine=True)
df.display()

In [0]:
from pyspark.sql.functions import *
df=df.withColumn("start_date",lit(start_date))
df=df.withColumn("end_date",lit("2099-12-31"))
df=df.withColumn("current_flag",lit("y"))

In [0]:
df.display()

In [0]:
df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.new_customer1")

In [0]:
df=spark.table("default.new_customer1")
df.display()

In [0]:
customer_data2=f"/Volumes/workspace/default/scd/Customer/customer_{start_date}.csv"
print(customer_data2)
df1=spark.read.csv(customer_data2,header=True,inferSchema=True,multiLine=True)
df1.display()

In [0]:
from pyspark.sql.functions import *
df1=df1.withColumn("start_date",lit(start_date))
df1=df1.withColumn("end_date",lit("2099-12-31"))
df1=df1.withColumn("current_flag",lit("y"))

In [0]:
df1.display()

In [0]:
from delta.tables import DeltaTable
delta_table = DeltaTable.forName(spark, "default.new_customer1")


In [0]:

delta_table.alias("target").merge(
    source=df1.select("customer_id","start_date").alias("source"),
   condition= "target.customer_id = source.customer_id AND target.current_flag='y'"
).whenMatchedUpdate(set={
     "end_date":"source.start_date",
     "current_flag":"'N'"}) \
 .execute()

In [0]:
df5=spark.table("default.new_customer1")
df5.display()

In [0]:
df1.write\
    .format("delta")\
    .mode("append")\
    .saveAsTable("default.new_customer1")

In [0]:
df6=spark.table("default.new_customer")
df6.display()
df6.printSchema()

In [0]:
customer_data=f"/Volumes/workspace/default/scd/Customer/customer_{start_date}.csv"
print(customer_data)
df2=spark.read.csv(customer_data,header=True,inferSchema=True,multiLine=True)
df2.display()
df2=df2.withColumn("phone_number",col("phone_number").cast("string"))

In [0]:
from pyspark.sql.functions import *
df2=df2.withColumn("start_date",lit(start_date))
df2=df2.withColumn("end_date",lit("2099-12-31"))
df2=df2.withColumn("current_flag",lit("y"))

In [0]:
df2.display()

In [0]:
delta_table.alias("target").merge(
    source=df2.select("customer_id","start_date").alias("source"),
   condition= "target.customer_id = source.customer_id AND target.current_flag='y'"
).whenMatchedUpdate(set={
     "end_date":"source.start_date",
     "current_flag":"'N'"}) \
 .execute()

In [0]:
df=spark.table("default.new_customer1")
df.display()

In [0]:
df2.write\
    .format("delta")\
    .mode("append")\
    .saveAsTable("default.new_customer1")

In [0]:
df=spark.table("default.new_customer1")
df.display()